In [ ]:
import xarray as xr
import numpy as np
import dask.array as da
import matplotlib.pyplot as plt
import os
from dask.distributed import Client, LocalCluster
from datetime import datetime,timedelta
import rioxarray
import xarray as xr
import glob
import rioxarray as rio
import cftime
import indices_function as ifun
from rechunker import rechunk


In [ ]:
cluster = LocalCluster(
    n_workers=20, 
    threads_per_worker=1,
    timeout='3600s',
    memory_limit='40GB',   
)
client = Client(cluster)
client

In [ ]:
variables=['tasmax','pr','tasmin']
for var in variables[1:]:
    datasets=[]
# List of file names
    for year in range(1990,2015):
        X1=xr.open_zarr("/nobackupp28/skhajehe/hawaii_multi/"+var+"/"+str(year)+".zarr",consolidated=False)
        datasets.append(X1)
    combined_data = xr.concat(datasets, dim='time')
    # pr_annual_proj=combined_data.chunk(dict(time=-1,lat=200,lon=200)).groupby('time.season').mean(dim='time')
    # pr_annual_proj.to_zarr("/nobackupp28/skhajehe/dcp-indices/multimodel/seasonal_average/"+var+"/2050.zarr",mode="w")
    precip = combined_data[var]  # Replace with your actual DataArray variable name

    # Define a helper function to classify months into summer/winter seasons
    def assign_season(month):
        # Return "summer" for May to October
        # Return "winter" for November to April
        if month in [5, 6, 7, 8, 9, 10]:
            return 'summer'
        else:
            return 'winter'

    # Create a season coordinate for each time step:
    # The `.dt.month` accessor returns the month number of each time value.
    month_numbers = combined_data['time'].dt.month
    season_values = xr.apply_ufunc(
        np.vectorize(assign_season), 
        month_numbers, 
        vectorize=True, 
        dask='parallelized' if precip.chunks else 'forbidden'
    )

    # Assign the new season coordinate to the dataset
    combined_data = combined_data.assign_coords(season=('time', season_values.values))

    # Now you can group by season and compute statistics:
    seasonal_mean = combined_data.groupby('season').mean(dim='time')
    seasonal_mean.to_zarr("/nobackupp28/skhajehe/hawaii_multi/"+var+"/seasonal.zarr",mode='w')

In [ ]:
# variables=['tasmax','pr','tasmin']
# for var in variables[1:]:
# # List of file names
    
reanalysis=xr.open_zarr("/nobackupp28/skhajehe/hawaii_rea/hawaii_reanalysis_max.zarr")
reanalysis = reanalysis.rename({"__xarray_dataarray_variable__": "tasmax"})
# Define a helper function to classify months into summer/winter seasons
def assign_season(month):
    # Return "summer" for May to October
    # Return "winter" for November to April
    if month in [5, 6, 7, 8, 9, 10]:
        return 'summer'
    else:
        return 'winter'

# Create a season coordinate for each time step:
# The `.dt.month` accessor returns the month number of each time value.
month_numbers = reanalysis['time'].dt.month
season_values = xr.apply_ufunc(
    np.vectorize(assign_season), 
    month_numbers, 
    vectorize=True, 
    dask='parallelized' if precip.chunks else 'forbidden'
)

# Assign the new season coordinate to the dataset
reanalysis = reanalysis.assign_coords(season=('time', season_values.values))

# Now you can group by season and compute statistics:
seasonal_mean = reanalysis.groupby('season').mean(dim='time')
seasonal_mean.to_zarr("/nobackupp28/skhajehe/hawaii_rea/seasonal_tasmax.zarr",mode='w')

In [ ]:
variables=['tasmax','pr','tasmin']
for var in variables:
    datasets=[]
# List of file names
    for year in range(1990,2015):
        X1=xr.open_zarr("/nobackupp28/skhajehe/hawaii_multi/"+var+"/"+str(year)+".zarr",consolidated=False)
        datasets.append(X1)
    combined_data = xr.concat(datasets, dim='time')
    # pr_annual_proj=combined_data.chunk(dict(time=-1,lat=200,lon=200)).groupby('time.season').mean(dim='time')
    # pr_annual_proj.to_zarr("/nobackupp28/skhajehe/dcp-indices/multimodel/seasonal_average/"+var+"/2050.zarr",mode="w")
    pr_annual_proj=combined_data.chunk(dict(time=-1,lat=200,lon=200)).mean(dim='time')
    pr_annual_proj.to_zarr("/nobackupp28/skhajehe/hawaii_multi/"+var+"/annual.zarr",mode="w")